In [1]:
import streamlit as st
import time
import numpy as np

import snowflake.connector
import os
import pandas as pd
from pathlib import Path

In [2]:
def connect():
    
    R_SNOFLK_USER = os.getenv('R_SNOFLK_USER')

    snowflake.connector.paramstyle = 'pyformat'

    conn = snowflake.connector.connect(
        user=R_SNOFLK_USER,
        password='N/A',
        account='ssq.canada-central.azure',
        authenticator='externalbrowser'
        )
    
    res_connect = conn.cursor()

    return res_connect

cs = connect()

2022-12-24 11:46:00.745 INFO    snowflake.connector.connection: Snowflake Connector for Python Version: 2.8.3, Python Version: 3.10.8, Platform: Windows-10-10.0.19044-SP0
2022-12-24 11:46:00.750 INFO    snowflake.connector.connection: This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
2022-12-24 11:46:00.750 INFO    snowflake.connector.connection: Setting use_openssl_only mode to False


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [3]:
date_debut = '2021-12-31'
date_fin = '2022-11-27'
date_debut_vie_ssq = 20211231
date_fin_vie_ssq = 20221127

In [4]:
cs.execute('USE DATABASE M_ASSURANCE_INDIVIDUELLE')
cs.execute('USE SCHEMA ACCES_BII_DATM')

query_LC_epargne = """\
SELECT  VENTE.DATE_PROD,
        PRODC.PRODC_NUMBR AS NO_OIPA,
        sum(vente.MNT_NOVL_ARGNT + vente.MNT_NOVL_RVD_EMIS) AS ENTREES
        
FROM    VENTE

JOIN PRODC ON VENTE.ID_CONS=PRODC.ID_PRODC

WHERE   date(DATE_PROD) > %(debut)s AND
        date(DATE_PROD) < %(fin)s
        
GROUP BY VENTE.DATE_PROD,
        PRODC.PRODC_NUMBR;"""

# df_epargne_LC = pd.read_sql(query_LC_epargne, conn)


cs.execute(query_LC_epargne, {"debut": date_debut, "fin": date_fin})

df_epargne_LC = pd.DataFrame(cs.fetch_pandas_all())

df_epargne_LC['NO_OIPA'] = df_epargne_LC['NO_OIPA'].str[4:10]

df_epargne_LC.dropna(axis=0, how='any', inplace=True)

df_epargne_LC['ENTREES'] = df_epargne_LC['ENTREES'].astype(float)


2022-12-24 11:49:08.216 INFO    snowflake.connector.cursor: query: [USE DATABASE M_ASSURANCE_INDIVIDUELLE]
2022-12-24 11:49:09.331 INFO    snowflake.connector.cursor: query execution done
2022-12-24 11:49:09.336 INFO    snowflake.connector.cursor: Number of results in first chunk: 1
2022-12-24 11:49:09.340 INFO    snowflake.connector.cursor: query: [USE SCHEMA ACCES_BII_DATM]
2022-12-24 11:49:09.499 INFO    snowflake.connector.cursor: query execution done
2022-12-24 11:49:09.504 INFO    snowflake.connector.cursor: Number of results in first chunk: 1
2022-12-24 11:49:09.506 INFO    snowflake.connector.cursor: query: [SELECT  VENTE.DATE_PROD, PRODC.PRODC_NUMBR AS NO_OIPA, sum(vente.MNT_NOVL_ARGNT ...]
2022-12-24 11:49:27.787 INFO    snowflake.connector.cursor: query execution done
2022-12-24 11:49:27.790 INFO    snowflake.connector.cursor: Number of results in first chunk: 2613


# LIFE

In [6]:
query_LC_vie = """\
SELECT  PRODC.PRODC_NUMBR AS NO_OIPA,
        sum(vente.MNT_PRIM_EMIS + vente.MNT_INV_PRIM_EMIS) AS PRIME_EMIS
        
FROM    VENTE

JOIN PRODC ON VENTE.ID_CONS=PRODC.ID_PRODC

WHERE   date(DATE_PROD) > %(debut)s AND
        date(DATE_PROD) < %(fin)s
        
GROUP BY PRODC.PRODC_NUMBR;"""

cs.execute(query_LC_vie, {"debut": date_debut, "fin": date_fin})

df_vie_LC = pd.DataFrame(cs.fetch_pandas_all())

df_vie_LC['NO_OIPA'] = df_vie_LC['NO_OIPA'].str[4:10]

df_vie_LC.dropna(axis=0, how='any', inplace=True)

df_vie_LC['PRIME_EMIS'] = df_vie_LC['PRIME_EMIS'].astype(float)

2022-12-20 10:10:38.946 INFO    snowflake.connector.cursor: query: [SELECT  PRODC.PRODC_NUMBR AS NO_OIPA, sum(vente.MNT_PRIM_EMIS + vente.MNT_INV_PR...]
2022-12-20 10:10:54.747 INFO    snowflake.connector.cursor: query execution done
2022-12-20 10:10:54.748 INFO    snowflake.connector.cursor: Number of results in first chunk: 3001


# Merge

In [23]:
merged_LC = pd.merge(df_epargne_LC, df_vie_LC, on="NO_OIPA", how="outer")
all_columns = tuple(merged_LC.columns)
my_columns = [1, 2]
columns = [all_columns[i] for i in my_columns]
columns

['ENTREES', 'PRIME_EMIS']

In [8]:
merged_LC

,NO_OIPA,ENTREES,PRIME_EMIS
0,129888,150914.94,0.00
1,123843,9730.00,0.00
2,124686,42515.00,0.00
3,660068,1097089.80,53293.41
4,129367,48616.00,2099.79
...,...,...,...
4375,660389,80.00,0.00
4376,136033,0.00,0.00
4377,136711,0.00,0.00
4378,137364,0.00,607.50


In [9]:
columns = tuple(merged_LC.columns)
print(columns)

('NO_OIPA', 'ENTREES', 'PRIME_EMIS')


In [16]:
df_epargne = merged_LC[["NO_OIPA", "ENTREES"]].copy()
df_epargne.sort_values(by="ENTREES", ascending=False)
df_epargne.head(20)

,NO_OIPA,ENTREES
0,129888,150914.94
1,123843,9730.00
2,124686,42515.00
3,660068,1097089.80
4,129367,48616.00
5,133733,1775.00
6,127246,4835.00
7,127230,1039219.30
8,115782,33785.00
9,502242,333961.97


In [25]:
def fetch_data():
    
    df_vie = merged_LC[["NO_OIPA", "PRIME_EMIS"]].copy()
    
    return df_vie

df_vie_output = fetch_data()

In [27]:
df_vie_output

,NO_OIPA,PRIME_EMIS
0,129888,0.00
1,123843,0.00
2,124686,0.00
3,660068,53293.41
4,129367,2099.79
...,...,...
4375,660389,0.00
4376,136033,0.00
4377,136711,0.00
4378,137364,607.50


[1, 5, 5]

In [34]:
df_epargne_LC['NO_OIPA']

0       129594
1       124274
2       570169
3       125256
4       117853
         ...  
4376    136252
4377    133908
4378    136033
4379    136711
4380    136991
Name: NO_OIPA, Length: 4380, dtype: object

In [14]:
df_epargne_LC.dtypes

DATE_PROD    datetime64[ns]
NO_OIPA              object
ENTREES             float64
dtype: object

In [13]:
df_epargne_LC["DATE_PROD"] = pd.to_datetime(df_epargne_LC["DATE_PROD"])

NameError: name 'datetime' is not defined